In [ ]:
#
#    Notebook de cours AMS-X02 - Cours 2 - M. Massot 2020-2021 - Ecole polytechnique
#    ----------   
#    Systèmes conservatifs
#    
#    Auteurs : L. Séries et M. Massot - (C) 2021
#    

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import root
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "seaborn"

# Systèmes conservatifs

## Mécanique céleste : problème à trois corps

On considère le système composé de trois corps : le Soleil, Jupiter et Saturne, les trois astres les plus massifs du système solaire. On fait l’hypothèse que le système est isolé et que les seules forces qui s’exercent sont les forces de gravitation entre les trois corps. 

Les masses respectives des trois corps soit noteés $m_i$, $i\in{0,1,2}$ et $G$ désigne la constante universelle de gravitation. On note $p_i(t)\in R^3$, $i\in{0,1,2}$ les quantités de mouvement de chaque astre. 

Le second principe de Newton pour chaque astre s'écrit :

$$
{\mathrm d}_t p_0 = \vec F_{Sa\rightarrow S} + \vec F_{J\rightarrow S},\quad
{\mathrm d}_t p_1 = \vec F_{S\rightarrow J} + \vec F_{Sa\rightarrow J} ,\quad
{\mathrm d}_t p_2 = \vec F_{S\rightarrow Sa} + \vec F_{J\rightarrow Sa},
$$

La force gravitationnelle $\vec F_{S\rightarrow P}$ exercée par un corps $S$ vers un corps $P$ est donné par la formule suivante :

$$
\vec F_{S\rightarrow P} = - \vec F_{P\rightarrow S} = - \frac{G\,m_S\,m_P}{d^2} \vec{u}, 
$$

où $d$ est la distance entre $S$ à $P$ et $\vec{u}$ un vecteur de longueur 1 dirigé de S vers P. 

In [ ]:
class three_body_model:

    def __init__(self):
        self.m0 = 1.00000597682
        self.m1 = 9.54786104043e-4
        self.m2 = 2.85583733151e-4
        self.G = 2.95912208286e-4

    def fcn(self, t, y):
        q = y[0:9]
        p = y[9:18]
        q_dot = self.H_p(p)
        p_dot = -self.H_q(q)
        return np.concatenate((q_dot, p_dot))

    def H_p(self, p):
        m0 = self.m0
        m1 = self.m1
        m2 = self.m2
        Hp = np.zeros(9)
        Hp[0:3] = p[0:3]/m0
        Hp[3:6] = p[3:6]/m1
        Hp[6:9] = p[6:9]/m2
        return Hp

    def H_q(self, q):
        m0 = self.m0
        m1 = self.m1
        m2 = self.m2
        G = self.G
        Hq = np.zeros(9)
        Hq[0:3] = ( G*m0*m1*((q[0:3]-q[3:6])/np.power(np.linalg.norm(q[0:3]-q[3:6]),3))
                   +G*m0*m2*((q[0:3]-q[6:9])/np.power(np.linalg.norm(q[0:3]-q[6:9]),3)))
        Hq[3:6] = ( G*m1*m0*((q[3:6]-q[0:3])/np.power(np.linalg.norm(q[3:6]-q[0:3]),3))
                   +G*m1*m2*((q[3:6]-q[6:9])/np.power(np.linalg.norm(q[3:6]-q[6:9]),3)))
        Hq[6:9] = ( G*m2*m0*((q[6:9]-q[0:3])/np.power(np.linalg.norm(q[6:9]-q[0:3]),3))
                   +G*m2*m1*((q[6:9]-q[3:6])/np.power(np.linalg.norm(q[6:9]-q[3:6]),3)))
        return Hq
    
    def hamiltonian(self, y):
    
        m0 = self.m0
        m1 = self.m1
        m2 = self.m2
        G = self.G
        nt = y.shape[1]
        neq = y.shape[0]
        q = y[0:neq//2]
        p = y[neq//2:neq]
        ham = np.zeros(nt)

        for i in range(nt):
            ham[i] = ( 0.5*( (1/m0)*np.dot(p[0:3,i],p[0:3,i]) 
                            +(1/m1)*np.dot(p[3:6,i],p[3:6,i]) 
                            +(1/m2)*np.dot(p[6:9,i],p[6:9,i]) ) 
                      - G *( (m0*m1)/np.linalg.norm(q[0:3,i]-q[3:6,i]) 
                            +(m0*m2)/np.linalg.norm(q[0:3,i]-q[6:9,i])
                            +(m1*m2)/np.linalg.norm(q[3:6,i]-q[6:9,i]) ) )

        return ham

### Intégration numérique 

In [ ]:
class ode_result:
    def __init__(self, y, t):
        self.y = y
        self.t = t

def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt))
    y[:,0] = yini

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*np.atleast_1d(fcn(tn, yn))

    return ode_result(y, t)

def backward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini

    def g(uip1, *args):
        uip, tip1 = args
        return uip1 - uip - dt*np.array(fcn(tip1, uip1))

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y0 = yn + dt*np.atleast_1d(fcn(tn, yn))
        # solve y[:,it+1] - y[:,it] - dt * fcn(tini + (it+1)*dt, y[:,it+1]) = 0
        sol = root(g, y0, (yn, tn+dt))
        y[:,it+1] = sol.x

    return ode_result(y, t)

def symplectic_euler(tini, tend, nt, yini, H_p, H_q):
    
    dt = (tend-tini) / (nt-1) 
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini
    
    q, p = np.split(y, 2)
    for it in range(nt-1):
        q_n = q[:,it]
        p_n = p[:,it]
        q[:,it+1] = q_n + dt*H_p(p_n)
        p[:,it+1] = p_n - dt*H_q(q[:,it+1])

    return ode_result(y,t)

In [ ]:
# initialization
m1 = 9.54786104043e-4
m2 = 2.85583733151e-4

qini = np.zeros(9)
qini[0] =  0.
qini[1] =  0.
qini[2] =  0.
qini[3] = -3.5023653
qini[4] = -3.8169847
qini[5] = -1.5507963
qini[6] =  9.0755314
qini[7] = -3.0458353
qini[8] = -1.6483708

pini = np.zeros(9)
pini[0] =  0.
pini[1] =  0.
pini[2] =  0.
pini[3] =  0.00565429*m1
pini[4] = -0.00412490*m1
pini[5] = -0.00190589*m1
pini[6] =  0.00168318*m2
pini[7] =  0.00483525*m2
pini[8] =  0.00192462*m2

yini = np.concatenate((qini, pini))

tini = 0.
tend = 11000.
nt = 1501

tbm = three_body_model()
H_p = tbm.H_p
H_q = tbm.H_q
fcn = tbm.fcn

# forward Euler integration
sol_fe = forward_euler(tini, tend, nt, yini, fcn)
yfe = sol_fe.y

# backward Euler integration
sol_be = backward_euler(tini, 0.95*tend, nt, yini, fcn)
ybe = sol_be.y

# symplectic Euler integration
sol_se = symplectic_euler(tini, tend, nt, yini, H_p, H_q)
yse = sol_se.y

fig = go.Figure()
color = fig.layout.template.layout.colorway
fig.add_trace(go.Scatter(x=yfe[3]-yfe[0], y=yfe[4]-yfe[0], line_color=color[0], legendgroup='fe', 
                         legendgrouptitle_text="Euler explicite", name='Jupiter'))
fig.add_trace(go.Scatter(x=yfe[6]-yfe[0], y=yfe[7]-yfe[0], line_color=color[0], legendgroup='fe',
                         line_dash='dash', name='Saturne'))
fig.add_trace(go.Scatter(x=ybe[3]-ybe[0], y=ybe[4]-ybe[0], line_color=color[1], legendgroup='be', 
                         legendgrouptitle_text="Euler implicite", name='Jupiter'))
fig.add_trace(go.Scatter(x=ybe[6]-ybe[0], y=ybe[7]-ybe[0], line_color=color[1], legendgroup='be',
                         line_dash='dash', name='Saturne'))
fig.add_trace(go.Scatter(x=yse[3]-yse[0], y=yse[4]-yse[0], line_color=color[2], legendgroup='se', 
                         legendgrouptitle_text="Euler symplectique", name='Jupiter'))
fig.add_trace(go.Scatter(x=yse[6]-yse[0], y=yse[7]-yse[0], line_color=color[2], legendgroup='se',
                         line_dash='dash', name='Saturne'))
fig.update_layout(legend=dict(groupclick="toggleitem"))
fig.show()


### Hamiltonien

Si on regarde l'évolution de l'énergie du système, on remarque que seul l'intégrateur symplectique conserve bien l'énegie du système.

In [ ]:
ham_fe = tbm.hamiltonian(sol_fe.y)
ham_be = tbm.hamiltonian(sol_be.y)
ham_se = tbm.hamiltonian(sol_se.y)

fig = go.Figure()
fig.add_trace(go.Scatter(x=sol_fe.t, y=ham_fe, line_color=color[0], name='Euler implicite'))
fig.add_trace(go.Scatter(x=sol_be.t, y=ham_be, line_color=color[1], name='Euler explicite'))
fig.add_trace(go.Scatter(x=sol_se.t, y=ham_se, line_color=color[2], name='Euler symplectique'))
fig.update_yaxes(exponentformat='e')
fig.update_layout(title="Hamiltonian")
fig.show()

## Arenstorf

Nous considérons un problème réduit à trois corps constitué du mouvement d'un satellite dans le cadre de l'attraction de la lune et de la terre. 
Pour les besoins de l'exercice, nous supposons que le système terre-lune est isolé et en rotation circulaire à vitesse constante dans un mouvement plan avec le centre de gravité de masse situé à l'origine, une solution classique d'une orbite périodique circulaire en mécanique céleste de deux corps isolés (appelé problème à deux corps classiquement).
Nous supposons aussi que la masse du satellite $\epsilon$ est suffisamment faible par rapport à la masse de la terre $1-\mu$ et la masse de la lune $\mu$ pour que nous puissions négliger son impact sur le système terre-lune.  Nous supposons également que le mouvement du satellite est régi par la loi de gravitation de Newton du fait de l'attraction des deux corps Terre et Lune.

Le mouvement du satellite dans le plan complexe satisfait l'équation : 

$$
\epsilon\, {\mathrm d}_t^2 Y = \frac{\epsilon(1-\mu)}{\vert A-Y \vert^2}\, \frac{A-Y}{\vert A-Y\vert}+ \frac{\epsilon\mu}{\vert B-Y\vert^2}\, \frac{B-Y}{\vert B-Y\vert},
$$

où le satellite est repéré par la coordonnée dans le plan complexe $Y(t)$ et $A$ et $B$ représentent la position de la terre et de la lune respectivement.

Pour éliminer le facteur $e^{it}$ dans $A=-\mu\,e^{it}$ et $A=(1-\mu)e^{it}$, on introduit la variable $y=e^{-it}\,Y = y_1+i\,y_2$. Dans ce nouveau référentiel tournant, la terre et la lune sont immobiles. Nous avons $Y=e^{it}y$ et  ${\mathrm d}_t^2 Y = -e^{it}y+2\,i\,e^{it}{\mathrm d}_t y+e^{it}{\mathrm d}_t^2 y$ et l'équation du mouvement s'écrit alors :

$$
{\mathrm d}_t^2 y + 2\,i\,{\mathrm d}_t y-y= (1-\mu)\,\frac{-\mu-y}{\vert  \mu+y\vert ^3}+ \mu\, \frac{1-\mu-y}{\vert  1-\mu-y \vert^3}.
$$

En introduisant les parties réelles et imaginaires de $y$ et en passant ensuite à un système d'équations différentielles de premier ordre, on obtient :

$$
\begin{array}{rcl} 
{\mathrm d}_t y_1 & = & y_3, \\ 
{\mathrm d}_t y_2 & = & y_4, \\ 
{\mathrm d}_t y_3 & = & y_1+2\,y_4- (1-\mu)(y_1+\mu)/r_1^3 - \mu (y_1-1+\mu)/r_2^3, \\
{\mathrm d}_t y_4 & = & y_2-2\,y_3- (1-\mu)y_2/r_1^3 - \mu y_2/r_2^3,
\end{array} 
$$

avec $r_1=((y_1+\mu)^2+y_2^2)^{1/2}$ and $((y_1-1+\mu)^2+y_2^2)^{1/2}$ et $\mu = 0.012277471$.

In [ ]:
class three_body_model:

    def __init__(self, mu):
        self.mu = mu

    def fcn(self, t, y) :
        y1,y2,y3,y4 = y
        mu = self.mu
        r1 = np.sqrt((y1+mu)*(y1+mu) + y2*y2)
        r2 = np.sqrt((y1-1+mu)*(y1-1+mu) + y2*y2)
        y1_dot = y3
        y2_dot = y4
        y3_dot = y1 + 2*y4 - (1-mu)*(y1+mu)/(r1*r1*r1) - mu*(y1 - 1 + mu)/(r2*r2*r2)
        y4_dot = y2 - 2*y3 - (1-mu)*y2/(r1*r1*r1) - mu*y2/(r2*r2*r2)
        return (y1_dot, y2_dot, y3_dot, y4_dot)

In [ ]:
# Initialization
yini = (-0.5655899165951338, 0.601315396569226, -0.45171183358756384, 0.23073427996775764)
#yini = (0.994, 0., 0., -2.00158510637908252240537862224)

tini = 0.
tend = 18.

mu = 0.012277471
    
tbm = three_body_model(mu)
fcn = tbm.fcn

###  Solution quasi-exacte

La solution quasi-exacte est obtenue par une méthode explicite de Runge-Kutta d'ordre(4)5 avec pas de temps adaptatives dans laquelle on fixe des tolérances très fines.

In [ ]:
tol = 1.e-12
sol = solve_ivp(fcn, (tini, tend), yini, rtol=tol, atol=tol)

fig_y1y2 = go.Figure()
fig_y1y2.add_trace(go.Scatter(x=sol.y[0], y=sol.y[1], name="quasi-exact sol.", showlegend=True))
fig_y1y2.add_trace(go.Scatter(x=[mu], y=[0.], mode="markers", marker=dict(size=10, color="black"), name="Terre"))
fig_y1y2.add_trace(go.Scatter(x=[1-mu], y=[0], mode="markers", marker=dict(size=6, color="brown"), name="Lune"))
fig_y1y2.update_layout(title="Arenstorf orbit", xaxis_title="y1", yaxis_title="y2")
fig_y1y2.show()

fig_sol = go.Figure()
fig_sol.add_trace(go.Scatter(x=sol.t, y=sol.y[0], name="y1"))
fig_sol.add_trace(go.Scatter(x=sol.t, y=sol.y[1], name="y2"))
fig_sol.add_trace(go.Scatter(x=sol.t, y=sol.y[2], name="y3"))
fig_sol.add_trace(go.Scatter(x=sol.t, y=sol.y[3], name="y4"))
fig_sol.update_layout(title="Evolution of solutions", xaxis_title="t")
fig_sol.show()

### Euler explicite

In [ ]:
tini = 0.
tend = 18.
nt = 180000
sol_fe = forward_euler(tini, tend, nt, yini, fcn)

fig = go.Figure()
fig.add_trace(go.Scatter(x=sol.y[0], y=sol.y[1], name="quasi-exact sol."))
fig.add_trace(go.Scatter(x=sol_fe.y[0], y=sol_fe.y[1], name="forward Euler"))
fig.add_trace(go.Scatter(x=[mu], y=[0.], mode="markers", marker=dict(size=10, color="black"), name="Terre"))
fig.add_trace(go.Scatter(x=[1-mu], y=[0], mode="markers", marker=dict(size=6, color="brown"), name="Lune"))
fig.update_layout(title="Arenstorf orbit", xaxis_title="y1", yaxis_title="y2")
fig.show()

### Euler implicite

In [ ]:
tini = 0.
tend = 18.
nt = 180000
sol_be = backward_euler(tini, tend, nt, yini, fcn)

fig = go.Figure()
fig.add_trace(go.Scatter(x=sol.y[0], y=sol.y[1], name="quasi-exact sol."))
fig.add_trace(go.Scatter(x=sol_be.y[0], y=sol_be.y[1], name="backward Euler"))
fig.add_trace(go.Scatter(x=[mu], y=[0.], mode="markers", marker=dict(size=10, color="black"), name="Terre"))
fig.add_trace(go.Scatter(x=[1-mu], y=[0], mode="markers", marker=dict(size=6, color="brown"), name="Lune"))
fig.update_layout(title="Arenstorf orbit", xaxis_title="y1", yaxis_title="y2")
fig.show()